In [12]:
import pandas as pd
from tqdm import tqdm
import time

In [13]:
# url = "https://race.netkeiba.com/race/result.html?race_id=202105010101"
# pd.read_html(url)[0]

In [14]:
pd.set_option('display.max_rows',20)
pd.options.display.width = 200
pd.options.display.column_space = 100

In [15]:
race_result = pd.read_pickle("df.pickle")
print(type(race_result.index[0]))

"202101010101" in race_result.index

<class 'str'>


True

In [16]:
race_id_lists = []

for 年 in range(2021,2022):
  for 競馬場 in range(1,6): # 1,11
    for 回 in range(1,4): # 1,11
      for 回目 in range(1,5): # 1,11
        for レース回 in range(1,3): # 1,13
          race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
          race_id_lists.append(race_id)
# print(race_id_lists)

# 実行時間 72分
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    if _race_id in race_result.index:
      continue
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
      # print(race_result_dict[race_id])
      time.sleep(1)
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict
  
add_race_result = scrape_race_results(race_id_lists)

100%|██████████| 120/120 [00:09<00:00, 12.36it/s]


In [17]:
len(add_race_result)

1

In [18]:
add_race_result

{'202101020401':     着順  枠  馬番         馬名  性齢    斤量    騎手     タイム     着差  人気  単勝オッズ   後3F   コーナー通過順     厩舎   馬体重(増減)
 0    1  2   2  イエヴァンポルッカ  牡2  54.0    坂井  1:31.6    NaN   3    3.9  34.8  11-10-11  美浦伊藤圭   470(+2)
 1    2  6   8  テイエムケントオー  牡2  52.0  △秋山稔  1:31.8  1.1/4   9   44.7  35.6     3-3-3  栗東五十嵐   446(-4)
 2    3  8  13    シーヴィクセン  牝2  54.0   ルメー  1:31.8     クビ   1    3.5  35.4     5-6-5  美浦藤沢和    474(0)
 3    4  7  11       ナバロン  牡2  54.0    菱田  1:31.9    1/2   4    9.9  35.8     2-2-2  栗東杉山佳  484(+10)
 4    5  4   5      エクセトラ  牡2  54.0    団野  1:32.0    アタマ   5   10.8  35.4     8-8-7   栗東吉村   500(+4)
 5    6  6   9   シュシュエラブル  牝2  51.0  ▲小林凌  1:32.0    アタマ  10  101.0  36.1     1-1-1   美浦田村   438(+6)
 6    7  7  10     チャオベッラ  牝2  51.0   ▲小沢  1:32.1      1   6   19.3  35.8     5-5-5   栗東西村  450(+12)
 7    8  8  12    グランジャンブ  牡2  54.0     黛  1:32.2     クビ  13  286.4  35.6     8-8-7  美浦鈴木慎  474(+10)
 8    9  4   4      キントリヒ  牡2  53.0   ☆泉谷  1:32.2    アタマ   7   22.5  35.1  

In [19]:
for key in add_race_result.keys():
  add_race_result[key].index = [key] * len(add_race_result[key])
print(type(add_race_result))

<class 'dict'>


In [20]:
results = pd.concat([add_race_result[key] for key in add_race_result.keys()],sort=False)
print(type(results))
results

<class 'pandas.core.frame.DataFrame'>


,着順,枠,馬番,馬名,性齢,斤量,騎手,タイム,着差,人気,単勝オッズ,後3F,コーナー通過順,厩舎,馬体重(増減)
202101020401,1,2,2,イエヴァンポルッカ,牡2,54.0,坂井,1:31.6,NaN,3,3.9,34.8,11-10-11,美浦伊藤圭,470(+2)
202101020401,2,6,8,テイエムケントオー,牡2,52.0,△秋山稔,1:31.8,1.1/4,9,44.7,35.6,3-3-3,栗東五十嵐,446(-4)
202101020401,3,8,13,シーヴィクセン,牝2,54.0,ルメー,1:31.8,クビ,1,3.5,35.4,5-6-5,美浦藤沢和,474(0)
202101020401,4,7,11,ナバロン,牡2,54.0,菱田,1:31.9,1/2,4,9.9,35.8,2-2-2,栗東杉山佳,484(+10)
202101020401,5,4,5,エクセトラ,牡2,54.0,団野,1:32.0,アタマ,5,10.8,35.4,8-8-7,栗東吉村,500(+4)
202101020401,6,6,9,シュシュエラブル,牝2,51.0,▲小林凌,1:32.0,アタマ,10,101.0,36.1,1-1-1,美浦田村,438(+6)
202101020401,7,7,10,チャオベッラ,牝2,51.0,▲小沢,1:32.1,1,6,19.3,35.8,5-5-5,栗東西村,450(+12)
202101020401,8,8,12,グランジャンブ,牡2,54.0,黛,1:32.2,クビ,13,286.4,35.6,8-8-7,美浦鈴木慎,474(+10)
202101020401,9,4,4,キントリヒ,牡2,53.0,☆泉谷,1:32.2,アタマ,7,22.5,35.1,13-13-12,美浦矢野,450(-2)
202101020401,10,5,6,オクシデンタル,牡2,54.0,吉田隼,1:32.2,クビ,8,43.2,36.0,3-3-3,美浦加藤征,502(+4)


In [21]:
results.to_pickle("add.results.pickle")